### rapyuta.io SDK turtlesim example.
Deploy turtlesim tutorial from SDK.

turtlesim tutorial:
SDK tutorial: https://docs.rapyuta.io/python-sdk/

#### 1.IDs
you can copy from rapyuta.io console

In [ ]:
AUTH_TOKEN = ''
PROJECT_ID = ''
SIMULATOR_PACKAGE_ID = ''
SIMULATOR_PLAN_ID = ''
COMMAND_CENTER_PACKAGE_ID = ''
COMMAND_CENTER_PLAN_ID = ''
TURTLE_PACKAGE_ID = ''
TURTLE_PLAN_ID = ''
UI_PACKAGE_ID = ''
UI_PLAN_ID = ''
DEPLOYMENT_PREFIX = 'turtlesim_'
TURTLE_NUM = 2

#### 2. import module and create Client Instance

In [ ]:
from rapyuta_io import Client
client = Client(AUTH_TOKEN, PROJECT_ID)

#### 3.Simulator deployment

In [ ]:
simulator_package = client.get_package(SIMULATOR_PACKAGE_ID)
simulator_configuration = simulator_package.get_provision_configuration(SIMULATOR_PLAN_ID)
simulator_deployment = simulator_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'simulator',
				provision_configuration = simulator_configuration)
simulator_deployment.poll_deployment_till_ready()

#### 4.Command Center deployment

In [ ]:
command_center_package = client.get_package(COMMAND_CENTER_PACKAGE_ID)
command_center_configuration = command_center_package.get_provision_configuration(COMMAND_CENTER_PLAN_ID)
command_center_configuration.add_dependent_deployment(simulator_deployment)
command_center_deployment = command_center_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'command_center',
				provision_configuration = command_center_configuration)
command_center_deployment.poll_deployment_till_ready()

#### 5.UI deployment

In [ ]:
ui_package = client.get_package(UI_PACKAGE_ID)
ui_configuration = ui_package.get_provision_configuration(UI_PLAN_ID)
ui_configuration.add_dependent_deployment(command_center_deployment)
ui_deployment = ui_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'ui',
				provision_configuration = ui_configuration)
result = ui_deployment.poll_deployment_till_ready()
print 'UI endpoint: ' + result['componentInfo'][0]['networkEndpoints']['UserInterface']

#### 6.Turtle deployments

In [ ]:
for i in range(TURTLE_NUM):
    turtle_package = client.get_package(TURTLE_PACKAGE_ID)
    turtle_configuration = turtle_package.get_provision_configuration(TURTLE_PLAN_ID)
    turtle_configuration.add_dependent_deployment(simulator_deployment)
    turtle_configuration.add_dependent_deployment(command_center_deployment)
    turtle_deployment = turtle_package.provision(
                    deployment_name = DEPLOYMENT_PREFIX + 'turtle' + str(i),
                    provision_configuration = turtle_configuration)
    turtle_deployment.poll_deployment_till_ready()
    print('Turtle' + str(i) + ' started')